In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from umap.umap_ import UMAP

In [2]:
#load al preprocess all data
soundData = sio.loadmat('data/soundData_small.mat')['sound_data_small'] #open data .mat file
# soundData =  pd.DataFrame(soundData)#get only actual data and convert to dataframe
soundData = (soundData-soundData.mean())/soundData.std()#normalisation

soundData_HL = sio.loadmat('data/soundData_HL_small.mat')['sound_data_small'] #open data .mat file
# soundData_HL =  pd.DataFrame(soundData_HL)#get only actual data and convert to dataframe
soundData_HL = (soundData_HL-soundData_HL.mean())/soundData_HL.std()#normalisation

soundData_CI = sio.loadmat('data/soundData_CI_small.mat')['sound_data_small'] #open data .mat file
# soundData_CI =  pd.DataFrame(soundData_CI)#get only actual data and convert to dataframe
soundData_CI = (soundData_CI-soundData_CI.mean())/soundData_CI.std()#normalisation

soundData_afterCI = sio.loadmat('data/soundData_afterCI_small.mat')['sound_data_small'] #open data .mat file
# soundData_afterCI =  pd.DataFrame(soundData_afterCI)#get only actual data and convert to dataframe
soundData_afterCI = (soundData_afterCI-soundData_afterCI.mean())/soundData_afterCI.std()#normalisation

In [3]:
# labels = pd.DataFrame(sio.loadmat('Stim288.mat')['labels'][0])
labels = sio.loadmat('Stim288.mat')['labels'][0]
# labels = labels.transpose()
pd.DataFrame(labels)

# soundLabels = pd.DataFrame(columns = ['label'])
# for label in labels:
#     soundLabels.loc[len(soundLabels.index)] = label#somehow the original array isnt good for svm, so i copy the data manually, idk why this does work
# soundLabels = np.asarray(soundLabels).ravel() #flatten the array so numpy is happy

#somehow the original array isnt good for svm, so i copy the data manually, idk why this does work
soundLabels=[]
for label in labels:
    soundLabels.append(label[0])
# print(np.asarray(labels))
print(soundLabels)

['speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'speech', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'voice', 'animal', 'animal', 'animal', 'animal', 'animal', 'animal', 'animal', 'animal', 'animal

In [15]:
umap_4d = UMAP(n_components=4, init='random', random_state=0)
proj_4d = umap_4d.fit_transform(soundData[:,0:14])
soundData_umap = np.hstack((soundData, proj_4d))

proj_4d_HL = umap_4d.fit_transform(soundData_HL[:,0:14])
soundData_umap_HL = np.hstack((soundData_HL, proj_4d_HL))

proj_4d_CI = umap_4d.fit_transform(soundData_CI[:,0:14])
soundData_umap_CI = np.hstack((soundData_CI, proj_4d_CI))

proj_4d_afterCI = umap_4d.fit_transform(soundData_afterCI[:,0:14])
soundData_umap_afterCI = np.hstack((soundData_afterCI, proj_4d_afterCI))

33408

In [7]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(soundData_umap, soundLabels, test_size=0.25, shuffle = True, stratify=soundLabels)
print(pd.DataFrame(y_train, columns = ["label"]).groupby(["label"]).size())

#create svm classifier
clf = SVC(kernel='rbf')

#fit svm on training data
clf.fit(x_train, y_train)

predictions = clf.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print("Healthy class Accuracy (normalized): " + str(accuracy))

label
animal    36
music     36
nature    36
speech    36
tools     36
voice     36
dtype: int64
Healthy class Accuracy (normalized): 0.2361111111111111


In [8]:
#k-fold cross validation
k = 4
kf = KFold(n_splits=k, random_state=None, shuffle = True)
result = cross_val_score(clf , soundData_umap, soundLabels, cv = kf)
 
print("Healthy class avg accuracy from CV: {}".format(result.mean()))

Healthy class avg accuracy from CV: 0.1701388888888889


In [9]:
#accuracy on other scenarios
#create svm classifier with all healthy data instead of train set
clf2 = clf
#fit svm on training data
clf2.fit(soundData_umap, soundLabels)

predictions_HL = clf2.predict(soundData_umap_HL)
accuracy_HL = accuracy_score(soundLabels, predictions_HL)
print("Hearing loss class Accuracy: " + str(accuracy_HL))

predictions_CI = clf2.predict(soundData_umap_CI)
accuracy_CI = accuracy_score(soundLabels, predictions_CI)
print("Cochlear implant class Accuracy: " + str(accuracy_CI))

predictions_afterCI = clf2.predict(soundData_umap_afterCI)
accuracy_afterCI = accuracy_score(soundLabels, predictions_afterCI)
print("Cochlear implant after Hearing loss class Accuracy: " + str(accuracy_afterCI))

Hearing loss class Accuracy: 0.21875
Cochlear implant class Accuracy: 0.22916666666666666
Cochlear implant after Hearing loss class Accuracy: 0.23958333333333334
